 Get Data from Wiki  importing required libraries to get data from wiki,   
 Importing BeautifulSoup to parse the html data,   
 Importing Pandas to work on the dataframe.  

In [2]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup

Getting the Data from wiki page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,"

In [4]:
data = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [5]:
output = data.read().decode()

Getting the Data using BeautifulSoup

In [6]:

soup = BeautifulSoup(output, 'html.parser')

Declaring Empty Dataframe with three columns 

In [7]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Reading Data from HTML tags and appending to DataFrame   
Assumptions - the table for PostalCode is the first table in the html page   
expected columns in the page in the order of Postcode, Borough , Neighbourhood  

In [8]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    #print(len(columns), columns)
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
        

In [9]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Finding the shape of the dataframe   
finding how many rows are with Borough as 'Not assigned', removing those rows  
and then checking the shape of the resulting dataframe

In [10]:
neighborhoods.shape

(289, 3)

289 Rows in the frame

In [11]:
neighborhoods['Borough'].value_counts()['Not assigned']

77

There are 77 rows where Borough is not Assigned - Filtering those rows

In [12]:
neighDF = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]

In [13]:
neighDF.shape

(212, 3)

'Not assgined' rows for Borough are filtered successfully 

In [14]:
neighDF.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Checking for rows where Neighbourhood is not assgined and Assigning the value of Borough to Neighborhood

In [15]:
neighDF['Neighborhood'].value_counts()['Not assigned']

1

only one such row found - let's see which rows are those

In [16]:
neighDF[neighDF['Neighborhood'].str.contains("Not assigned")==True]

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


row with Neighbourhood as 'Not assgined' is corresponding to "Queen's Park"

In [17]:
neighDF['Neighborhood'].replace(to_replace="Not assigned", value=neighDF['Borough'], inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [18]:
neighDF['Neighborhood'].value_counts().get('Not assgined',0)

0

In [19]:
neighDF['Neighborhood'].value_counts().get("Queen's Park",0)

1

Grouping the Values by Postcode and Borough and concatenating the text in Neighbourhood columns

check the shape of unique postcode and borugh values before grouping

In [20]:
len(neighDF['PostalCode'].unique()), len(neighDF['Borough'].unique())

(103, 11)

In [65]:
newDF = pd.DataFrame({'Neighborhood' : neighDF.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()

check the shape of unique postcode and borugh values after grouping

In [66]:
len(neighDF['PostalCode'].unique()), len(neighDF['Borough'].unique())

(103, 11)

In [67]:
newDF.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [68]:
newDF.shape

(103, 3)

Reading data from Geospatial_Data file downloaded from "http://cocl.us/Geospatial_data"

In [69]:
geoDF = pd.read_csv("Geospatial_data")

In [70]:
geoDF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Three columns in the dataframe merging them with the Toronto PostalCode data

In [90]:
result = pd.concat([newDF, geoDF], axis=1, )

In [94]:
result.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [96]:
resultDF = result.drop('Postal Code', axis = 1)

In [98]:
resultDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
